In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score
from scipy.stats import chi2_contingency

### Objective Function

##### Maximize IV value of the bins

IV = ∑((Good% - Bad%) * ln(Good% / Bad%)), where Good% and Bad% are the proportions of non-default and default observations, respectively, in each bin.

##### Constraints:

- The default rate per bin should be monotonically increasing or decreasing.
- The bins should be statistically different (determined by the Chi-square test or another suitable hypothesis test with a chosen significance level).
- The minimum number of observations per bin should be met.
- The minimum and maximum number of bins should be within the specified range.

##### Variables:

- Bins: A set of ordinal categories.
- Default rate: The proportion of default observations (default_flag = 1) in each bin.

##### Sequential Least Squares Quadratic Programming

SLSQP stands for Sequential Least Squares Quadratic Programming. It is an optimization algorithm available in the scipy.optimize module. The SLSQP method is a gradient-based optimization algorithm used for solving nonlinear optimization problems with equality and inequality constraints. It is particularly suitable for solving problems that have a smooth objective function and smooth constraints. The method takes an objective function, an initial guess for the solution, bounds on the variables, and constraints as input. It then tries to find the optimal solution that minimizes the objective function while satisfying the constraints.

Here's a brief overview of how the SLSQP algorithm works:

- The algorithm starts with an initial guess for the solution and calculates the objective function value, its gradient (first-order derivatives), and the constraint values and their gradients at this initial point.
- It then iteratively improves the solution by approximating the objective function and constraints with quadratic functions (hence the name Quadratic Programming) using the calculated gradient information. The algorithm solves a sequence of quadratic subproblems subject to linearized constraints to find a new solution.
- The new solution is checked for feasibility (i.e., whether it satisfies the constraints) and optimality (i.e., whether the objective function value has improved). If necessary, the algorithm adjusts the solution using a line search method to satisfy the constraints.
- The algorithm proceeds iteratively, updating the solution and recalculating the objective function, gradients, and constraint values until convergence is achieved, or a stopping criterion is met.

SLSQP is a good choice for problems with a moderate number of variables and constraints where the objective function and constraints are smooth (i.e., differentiable). However, it may not be the best choice for large-scale, non-smooth, or non-convex problems. In such cases, other optimization algorithms might be more suitable.

### A. Create test data

In [2]:
np.random.seed(42)

def generate_data(n):
    years = np.random.choice([2016, 2017, 2018, 2019, 2020, 2021], n)
    economic_cycle = np.random.rand(n)
    explanatory_variable = np.random.rand(n) * 10

    # Add noise to the relationship between explanatory_variable and economic_cycle
    noise = np.random.normal(0, 0.1, n)

    # Define a non-trivial and random relationship between target and explanatory variable
    z = (explanatory_variable / 10) * economic_cycle + noise
    target_proba = 1 / (1 + np.exp(-z))  # Apply logistic function
    target_variable = (target_proba > 0.5).astype(int)

    data = {
        'explanatory_variable': explanatory_variable,
        'target_variable': target_variable,
        'year': years,
        'economic_cycle': economic_cycle
    }

    return pd.DataFrame(data)

# Choose the number of observations
n = 200_000 

# Create dataset
df = generate_data(n)
print(df.head())

   explanatory_variable  target_variable  year  economic_cycle
0              6.678090                0  2019        0.140130
1              8.521948                1  2020        0.914430
2              4.038421                1  2018        0.783868
3              3.281745                1  2020        0.119363
4              8.264844                1  2020        0.568544


### B. Create optimal pooling/calibration option

In [3]:
def calculate_iv(df, bin_edges):
    binned_data = pd.cut(df['explanatory_variable'], bins=bin_edges, include_lowest=True)
    bin_summary = df.groupby(binned_data)['target_variable'].agg(['count', 'sum'])
    bin_summary['non_target'] = bin_summary['count'] - bin_summary['sum']
    
    bin_summary['target_dist'] = bin_summary['sum'] / bin_summary['sum'].sum()
    bin_summary['non_target_dist'] = bin_summary['non_target'] / bin_summary['non_target'].sum()
    
    bin_summary['woe'] = np.log(bin_summary['target_dist'] / bin_summary['non_target_dist'])
    bin_summary['iv'] = (bin_summary['target_dist'] - bin_summary['non_target_dist']) * bin_summary['woe']
    
    return bin_summary['iv'].sum()

def calculate_event_rates(df, bin_edges):
    annual_event_rates = []

    for year in df['year'].unique():
        df_year = df[df['year'] == year]
        bin_indices = np.digitize(df_year['explanatory_variable'], bin_edges)
        bin_counts = np.bincount(bin_indices)[1:len(bin_edges)]
        event_counts = np.bincount(bin_indices, weights=df_year['target_variable'])[1:len(bin_edges)]
        annual_event_rates.append(event_counts / bin_counts)

    average_annual_event_rates = np.mean(annual_event_rates, axis=0)
    return average_annual_event_rates

def optimize_iv(df, min_bins=2, max_bins=10, min_bin_size=0.01, monotonic=True):
    best_bin_edges = None
    best_objective_value = np.inf
    
    def iv_objective(bin_edges):
        extended_edges = [df['explanatory_variable'].min()] + list(bin_edges) + [df['explanatory_variable'].max()]
        print(f'Number of edges: {len(bin_edges)}, IV: {calculate_iv(df, extended_edges):.2%}')
        
        return -calculate_iv(df, extended_edges)

    def monotonic_constraint(bin_edges):
        extended_edges = [df['explanatory_variable'].min()] + list(bin_edges) + [df['explanatory_variable'].max()]
        event_rates = calculate_event_rates(df, extended_edges)
        
        if monotonic:
            if np.all(np.diff(event_rates) >= 0) or np.all(np.diff(event_rates) <= 0):
                return 0
            else:
                return np.inf
        else:
            return 0
        
    def bins_statistical_difference(bin_edges, df, significance_level=0.05):
        extended_edges = [df['explanatory_variable'].min()] + list(bin_edges) + [df['explanatory_variable'].max()]
        df['temp_bin'] = pd.cut(df['explanatory_variable'], bins=extended_edges, labels=False, include_lowest=True)

        for i in range(len(extended_edges) - 2):
            contingency_table = pd.crosstab(df[df['temp_bin'].isin([i, i + 1])]['temp_bin'], df['target_variable'])
            _, p, _, _ = chi2_contingency(contingency_table)
            # print(f'p-value: {p}')

            if p > significance_level:
                return np.inf

        return 0

    for num_bins in range(min_bins, max_bins + 1):
        initial_bin_edges = np.linspace(df['explanatory_variable'].min(), df['explanatory_variable'].max(), num_bins + 1)[1:-1]
        
        print(1, initial_bin_edges)
        
        constraints = [{'type': 'ineq', 'fun': monotonic_constraint},
                       {'type': 'ineq', 'fun': lambda x: bins_statistical_difference(x, df)}
                      ]
        bounds = [(min_bin_size, None) for _ in range(num_bins - 1)]

        result = minimize(iv_objective, initial_bin_edges, method='SLSQP', bounds=bounds, constraints=constraints)

        if result.success and -result.fun < best_objective_value:
            best_objective_value = -result.fun
            best_bin_edges = [df['explanatory_variable'].min()] + list(result.x) + [df['explanatory_variable'].max()]

    return best_bin_edges

optimal_bin_edges = optimize_iv(df, min_bins=10, max_bins=20)
print("Optimal bin edges:", optimal_bin_edges)

# Create a new column in the dataset that represents the bin assignments
df['bin'] = pd.cut(df['explanatory_variable'], bins=optimal_bin_edges, labels=False, include_lowest=True)

# Calculate ROC AUC scores
roc_auc_non_binned = roc_auc_score(df['target_variable'], df['explanatory_variable'])
roc_auc_binned = roc_auc_score(df['target_variable'], df['bin'])

print(f"ROC AUC score (non-binned): {roc_auc_non_binned:.4f}")
print(f"ROC AUC score (binned): {roc_auc_binned:.4f}")


1 [1.00002696 2.00002054 3.00001411 4.00000768 5.00000125 5.99999482
 6.99998839 7.99998196 8.99997553]
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
Number of edges: 9, IV: 79.83%
1 [0.90911846 1.81820352 2.72728859 3.63637365 4.54545871 5.45454378
 6.36362884 7.27271391 8.18179897 9.09088404]
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
Number of edges: 10, IV: 80.23%
1 [0.83336137 1.66668935 2.50001732 3.3333453  4.16667327 5.00000125
 5.83332922 6.6666572  7.49998517 8.33331315 9.166

C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f

1 [0.71431452 1.42859564 2.14287676 2.85715788 3.571439   4.28572013
 5.00000125 5.71428237 6.42856349 7.14284461 7.85712573 8.57140686
 9.28568798]
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%
Number of edges: 13, IV: 80.80%


C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f

1 [0.66669577 1.33335815 2.00002054 2.66668292 3.3333453  4.00000768
 4.66667006 5.33333244 5.99999482 6.6666572  7.33331958 7.99998196
 8.66664434 9.33330672]
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%
Number of edges: 14, IV: 80.96%


...

C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f

1 [0.50003018 1.00002696 1.50002375 2.00002054 2.50001732 3.00001411
 3.50001089 4.00000768 4.50000446 5.00000125 5.49999803 5.99999482
 6.4999916  6.99998839 7.49998517 7.99998196 8.49997874 8.99997553
 9.49997231]
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%
Number of edges: 19, IV: 81.39%


C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f

Optimal bin edges: [3.339410015512634e-05, 1.000026964687575, 2.000020535274995, 3.000014105862415, 4.000007676449835, 5.000001247037255, 5.999994817624675, 6.999988388212095, 7.9999819587995145, 8.999975529386935, 9.999969099974354]
ROC AUC score (non-binned): 0.7423
ROC AUC score (binned): 0.7387


C:\Users\Watson\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


### C. Implement using OR-tools

Consider starting with large number of bins and to use constraint programming to solve it.

In [4]:
from ortools.sat.python import cp_model
import numpy as np
import pandas as pd

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, num_bins, bin_vars):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._num_bins = num_bins
        self._bin_vars = bin_vars
        self._optimal_bin_edges = []

    def on_solution_callback(self):
        self._optimal_bin_edges = [self.Value(var) for var in self._bin_vars]

    def optimal_bin_edges(self):
        return self._optimal_bin_edges
    
    
def calculate_iv_based_on_bins(df, bins):
    binned_data = pd.cut(df['explanatory_variable'], bins=bins, include_lowest=True)
    bin_summary = df.groupby(binned_data)['target_variable'].agg(['count', 'sum'])
    bin_summary['non_target'] = bin_summary['count'] - bin_summary['sum']

    bin_summary['target_dist'] = bin_summary['sum'] / bin_summary['sum'].sum()
    bin_summary['non_target_dist'] = bin_summary['non_target'] / bin_summary['non_target'].sum()

    bin_summary['woe'] = np.log(bin_summary['target_dist'] / bin_summary['non_target_dist'])
    bin_summary['iv'] = (bin_summary['target_dist'] - bin_summary['non_target_dist']) * bin_summary['woe']

    return bin_summary['iv'].sum()

def optimize_iv_cp(df, num_bins):
    # Create the model.
    model = cp_model.CpModel()

    # Create the variables.
    bin_vars = [model.NewIntVar(df['explanatory_variable'].min(), df['explanatory_variable'].max(), f'bin_{i}') for i in range(num_bins - 1)]

    # Add the constraints.
    for i in range(num_bins - 2):
        model.Add(bin_vars[i] <= bin_vars[i + 1])

    # Define the objective.
    iv_values = [calculate_iv_based_on_bins(df, [df['explanatory_variable'].min()] + [var_value] + [df['explanatory_variable'].max()]) for var_value in range(df['explanatory_variable'].min(), df['explanatory_variable'].max())]
    objective = sum(iv_values[i] * (bin_vars[i] - bin_vars[i - 1]) for i in range(1, num_bins - 1))
    model.Maximize(objective)

    # Solve the model.
    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter(num_bins, bin_vars)
    status = solver.SearchForAllSolutions(model, solution_printer)

    if status == cp_model.OPTIMAL:
        return solution_printer.optimal_bin_edges()
    else:
        return None

num_bins = 250
optimal_bin_edges = optimize_iv_cp(df, num_bins)
print("Optimal bin edges:", optimal_bin_edges)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. ortools.util.python.sorted_interval_list.Domain(arg0: int, arg1: int)

Invoked with: 3.339410015512634e-05, 9.999969099974354

In [ ]:
xxxxxxxxxx

### This is just a test, disregard

In [ ]:
from ortools.linear_solver import pywraplp

def optimize_iv_ortools(df, min_bins=2, max_bins=10, min_bin_size=0.01, monotonic=True):
    best_bin_edges = None
    best_objective_value = np.inf
    
    num_bins = 250
    initial_bin_edges = np.linspace(df['explanatory_variable'].min(), df['explanatory_variable'].max(), num_bins + 1)[1:-1]

    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        raise ValueError('No solver available.')

    # Variables
    bin_edges = [solver.NumVar(df['explanatory_variable'].min(), df['explanatory_variable'].max(), f'bin_edge_{i}') for i in range(num_bins - 1)]

    # Objective
    iv_objective = solver.Objective()
    iv_objective.SetMinimization()
    for i, edge in enumerate(bin_edges):
        iv_objective.SetCoefficient(edge, -calculate_iv(df, [df['explanatory_variable'].min()] + list(initial_bin_edges[:i+1]) + [df['explanatory_variable'].max()]))

    # Constraints
    for i in range(num_bins - 2):
        solver.Add(bin_edges[i] <= bin_edges[i + 1])

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        best_objective_value = -solver.Objective().Value()
        best_bin_edges = [df['explanatory_variable'].min()] + [edge.solution_value() for edge in bin_edges] + [df['explanatory_variable'].max()]

    return best_bin_edges


optimal_bin_edges = optimize_iv_ortools(df, min_bins=3, max_bins=6)
# print("Optimal bin edges:", optimal_bin_edges)


# Create a new column in the dataset that represents the bin assignments
df['bin'] = pd.cut(df['explanatory_variable'], bins=optimal_bin_edges, labels=False, include_lowest=True, duplicates='drop')

# Calculate ROC AUC scores
roc_auc_non_binned = roc_auc_score(df['target_variable'], df['explanatory_variable'])
roc_auc_binned = roc_auc_score(df['target_variable'], df['bin'])

print(f"ROC AUC score (non-binned): {roc_auc_non_binned:.4f}")
print(f"ROC AUC score (binned): {roc_auc_binned:.4f}")



In [ ]:
xxxxxxxxxx

### Appendix

To determine whether adjacent bins are statistically different, you can perform a hypothesis test, such as the Chi-square test for independence. 

Here's a step-by-step process for doing this:
- Create a contingency table for the adjacent bins, showing the frequency distribution of the binary default flag (0 and 1) for each pair of adjacent bins.
- Calculate the expected frequency for each cell in the contingency table under the assumption that the bins are independent. To do this, multiply the row total (sum of binary default flags in a bin) by the column total (sum of observations in the two adjacent bins) and divide by the total number of observations.
- Calculate the Chi-square statistic. For each cell, find the difference between the observed frequency and the expected frequency, square the result, and divide by the expected frequency. Add up all these values to get the Chi-square statistic.
- Determine the degrees of freedom for the Chi-square test. For a 2x2 contingency table (2 rows, one for each binary default flag, and 2 columns, one for each adjacent bin), the degrees of freedom are (number of rows - 1) * (number of columns - 1) = 1.
- Determine the critical value and the p-value for the calculated Chi-square statistic using the Chi-square distribution table or a statistical software with the corresponding degrees of freedom.
- Set a significance level (commonly 0.05). If the p-value is less than the chosen significance level, reject the null hypothesis that the adjacent bins are independent, which means there is a statistically significant difference between the bins. If the p-value is greater than the chosen significance level, you cannot reject the null hypothesis, and there is no evidence to suggest a statistically significant difference between the bins.

Repeat these steps for each pair of adjacent bins to determine whether there is a statistically significant difference between them.

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Sample data
data = {
    'bin': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'E'],
    'default_flag': [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1]
}

df = pd.DataFrame(data)

# Perform Chi-square test for independence on adjacent bins
significance_level = 0.05

for i in range(len(df['bin'].unique()) - 1):
    bin1 = df['bin'].unique()[i]
    bin2 = df['bin'].unique()[i + 1]
    contingency_table = pd.crosstab(df[df['bin'].isin([bin1, bin2])]['bin'], df['default_flag'])
    chi2, p, _, _ = chi2_contingency(contingency_table)
    
    print(f"Chi-square test for {bin1} and {bin2}:")
    print(f"Chi2 = {chi2:.4f}, p-value = {p:.4f}")
    
    if p < significance_level:
        print(f"The difference between bins {bin1} and {bin2} is statistically significant.\n")
    else:
        print(f"There is no evidence to suggest a statistically significant difference between bins {bin1} and {bin2}.\n")


In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

data = np.array([
    [60, 30],
    [20, 50],
])

chi2, p_value, dof, expected = chi2_contingency(data)

print(chi2, p_value, dof, expected)